In [ ]:
import time

from src.swagger import SwaggerParser
from src.config import Config
import json
import os

os.environ["model"] = "gpt-4o-2024-05-13"
os.environ["language_model_key"] = ""

# root_path = "/Users/naariah/Documents/Python_Codes/api-suts/specifications/v3/languageTool.json"
root_path = "/Users/naariah/Documents/Python_Codes/api-suts/specifications/v3/gitlab-groups-13.json"
config = Config()
config.swagger = root_path
parser = SwaggerParser(config)
operations = parser.extract()

In [ ]:
operations

In [ ]:
from src.rest import PathParam

info = []
for operation in operations:
    info.append(
                {
                    "name": operation.__repr__(),
                    "description": operation.description,
                }
            )

In [ ]:
info

In [ ]:
operation = operations[3]
info = []
for f in operation.get_leaf_factors():
    info.append(
                {
                    "name": f.get_global_name,
                    "type": f.type.value,
                    "description": f.description,
                }
            )

In [ ]:
from openai import OpenAI
from src.template import *
from src.languagemodel.llm import count_tokens

client = OpenAI(api_key=os.environ["language_model_key"],base_url="https://api.bianxieai.com/v1")
messages = []
messages.append({"role": "system", "content": SystemRole.SYS_ROLE_SEQUENCE})
prompt = INFO.SEQUENCE.format(info) + Task.SEQUENCE
messages.append({"role": "user", "content": prompt})

In [ ]:
messages

In [ ]:
count_tokens(messages, os.environ["model"])

In [ ]:
from loguru import logger

while True:
    try:
        start = time.time()
        response = client.chat.completions.create(
            model=os.environ["model"],
            messages=messages,
            temperature=0.5,
            top_p=0.99,
            frequency_penalty=0,
            presence_penalty=0,
            max_tokens=4096,
            response_format={"type": "json_object"}
        )
        end = time.time()
        print("Time taken: ", end - start)
        break
    except Exception as e:
        logger.error(f"Error in calling language model: {e}")
        logger.error("Retrying...")

In [ ]:
json.loads(response.choices[0].message.content)

In [15]:
import rstr


def generate_random_string_from_pattern(pattern, min_length=0, max_length=None):
    try:
        generated_str = rstr.xeger(pattern)
        if max_length:
            generated_str = generated_str[min_length:max_length-1]
        return generated_str
    except Exception:
        pass
    return None


pattern = r'[A-Za-z0-9!@#$%^&*()_+]{10,15}'
random_string = generate_random_string_from_pattern(pattern)
print(random_string)

fxW7VndDBfq_TTr
